# Distributed Hypermeter Optimization

This example utilizes the Ray Framework to conduct distributed hyperparameter
optimization using the Bayesian Optimization HyperBand (BOHB) optimization
technique. Bayesian Optimization HyperBand (BOHB) combines the benefits of
Bayesian optimization together with Bandit-based methods (e.g. HyperBand). 
BOHB does not rely on the gradient of the objective function, but instead,
learns from samples of the search space. It is suitable for optimizing functions
that are non-differentiable, with many local minima, or even unknown but only
testable. Therefore, this approach belongs to the domain of "derivative-free
optimization" and "black-box optimization".

In this example we minimize a simple objective to briefly demonstrate the usage of
BOHB with Ray Tune via `BOHBSearch`. It's useful to keep in mind that despite
the emphasis on machine learning experiments, Ray Tune optimizes any implicit
or explicit objective. Here we assume `ConfigSpace`, `pyarrow`, and `hpbandster==0.7.4`
libraries are installed. To learn more, please refer to the
[BOHB website](https://github.com/automl/HpBandSter).

In [1]:
!pip install ray[tune]
!pip install ConfigSpace
!pip install pyarrow
!pip install hpbandster==0.7.4


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [2]:
import time

import ray
from ray import air, tune
from ray.air import session
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB
import ConfigSpace as CS

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune
two hyperparameters, namely `width` and `height`, and `activation`.

In [3]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our `objective` function takes a Tune `config`, evaluates the `score` of your
experiment in a training loop, and uses `session.report` to report the `score` back to Tune.

In [4]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})

In [5]:
ray.init(configure_logging=False)

Python version:,3.11.4
Ray version:,2.5.1


Next we define a search space. The critical assumption is that the optimal
hyperparameters live within this space. Yet, if the space is very large,
then those hyperparameters may be difficult to find in a short amount of time.

In [6]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"]),
}

Next we define the search algorithm built from `TuneBOHB`, constrained
to a maximum of `4` concurrent trials with a `ConcurrencyLimiter`.
Below `algo` will take care of the BO (Bayesian optimization) part of BOHB,
while scheduler will take care the HB (HyperBand) part.

In [7]:
algo = TuneBOHB()
algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=100,
    reduction_factor=4,
    stop_last_trials=False,
)

The number of samples is the number of hyperparameter combinations
that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [8]:
num_samples = 1000

In [9]:
num_samples = 10

Finally, we run the experiment to `min`imize the "mean_loss" of the `objective`
by searching within `"steps": 100` via `algo`, `num_samples` times. This previous
sentence is fully characterizes the search problem we aim to solve.
With this in mind, notice how efficient it is to execute `tuner.fit()`.

In [10]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=num_samples,
    ),
    run_config=air.RunConfig(
        name="bohb_exp",
        stop={"training_iteration": 100},
    ),
    param_space=search_space,
)
results = tuner.fit()

You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behavior. To avoid this, install fsspec with `pip install fsspec`. Depending on your remote storage provider, consider installing the respective fsspec-package (see https://github.com/fsspec).
The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A more strict API access pattern would be enforced starting 1.12s.0


Trial name,date,done,episodes_total,hostname,iterations,iterations_since_restore,mean_loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_total,training_iteration,trial_id
objective_084692a3,2024-04-09_11-46-40,False,0,Neils-MacBook-Air.local,6,7,4.08576,127.0.0.1,33324,0.718058,0.10165,1.24273,1712677600,0,7,084692a3
objective_28d984f3,2024-04-09_11-46-34,False,,Neils-MacBook-Air.local,0,1,17.7851,127.0.0.1,33302,0.100721,0.100721,0.100721,1712677594,,1,28d984f3
objective_34f36a0f,2024-04-09_11-46-41,True,0,Neils-MacBook-Air.local,15,16,1.36363,127.0.0.1,33326,1.73358,0.102987,2.24606,1712677601,0,16,34f36a0f
objective_3f997772,2024-04-09_11-46-34,False,,Neils-MacBook-Air.local,0,1,18.5748,127.0.0.1,33302,0.10332,0.10332,0.10332,1712677594,,1,3f997772
objective_3fa9fa28,2024-04-09_11-46-35,False,,Neils-MacBook-Air.local,0,1,9.78275,127.0.0.1,33302,0.104727,0.104727,0.104727,1712677595,,1,3fa9fa28
objective_91f765fd,2024-04-09_11-46-40,False,0,Neils-MacBook-Air.local,8,9,1.65683,127.0.0.1,33325,0.936717,0.101809,1.45188,1712677600,0,9,91f765fd
objective_a7c061e0,2024-04-09_11-46-52,True,0,Neils-MacBook-Air.local,99,100,-0.734633,127.0.0.1,33329,10.4619,0.105676,12.6643,1712677612,0,100,a7c061e0
objective_c0c2bbc5,2024-04-09_11-46-35,False,,Neils-MacBook-Air.local,0,1,25.3466,127.0.0.1,33302,0.104408,0.104408,0.104408,1712677595,,1,c0c2bbc5
objective_cf11b025,2024-04-09_11-46-34,False,,Neils-MacBook-Air.local,0,1,24.829,127.0.0.1,33302,0.105235,0.105235,0.105235,1712677594,,1,cf11b025
objective_de910ff8,2024-04-09_11-46-34,False,,Neils-MacBook-Air.local,0,1,20.9478,127.0.0.1,33302,0.105291,0.105291,0.105291,1712677594,,1,de910ff8


(objective pid=33315) 2024-04-09 11:46:36,754	INFO trainable.py:918 -- Restored on 127.0.0.1 from checkpoint: /Users/neil/ray_results/bohb_exp/objective_a7c061e0_1_activation=tanh,height=-18.1708,steps=100,width=12.1510_2024-04-09_11-46-33/checkpoint_tmped3fe4
(objective pid=33315) 2024-04-09 11:46:36,754	INFO trainable.py:927 -- Current state after restoring: {'_iteration': 0, '_timesteps_total': 0, '_time_total': 0.10368919372558594, '_episodes_total': 0}
(objective pid=33329) 2024-04-09 11:46:41,948	INFO trainable.py:918 -- Restored on 127.0.0.1 from checkpoint: /Users/neil/ray_results/bohb_exp/objective_a7c061e0_1_activation=tanh,height=-18.1708,steps=100,width=12.1510_2024-04-09_11-46-33/checkpoint_tmp9cab6d [repeated 14x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(objective pid=33329) 2024-04-09 11:46:41,948	INFO t

Here are the hyperparameters found to minimize the mean loss of the defined objective.

In [11]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 12.151027482869823, 'height': -18.17076073631992, 'activation': 'tanh'}


## Optional: Passing the search space via the TuneBOHB algorithm

We can define the hyperparameter search space using `ConfigSpace`,
which is the format accepted by BOHB.

In [ ]:
config_space = CS.ConfigurationSpace()
config_space.add_hyperparameter(
    CS.Constant("steps", 100)
)
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter("width", lower=0, upper=20)
)
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter("height", lower=-100, upper=100)
)
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        "activation", choices=["relu", "tanh"]
    )
)

In [ ]:
# As we are passing config space directly to the searcher,
# we need to define metric and mode in it as well, in addition
# to Tuner()
algo = TuneBOHB(
    space=config_space,
    metric="mean_loss",
    mode="max",
)
algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=100,
    reduction_factor=4,
    stop_last_trials=False,
)

In [ ]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=num_samples,
    ),
    run_config=air.RunConfig(
        name="bohb_exp_2",
        stop={"training_iteration": 100},
    ),
)
results = tuner.fit()

Here again are the hyperparameters found to minimize the mean loss of the
defined objective.

In [12]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 12.151027482869823, 'height': -18.17076073631992, 'activation': 'tanh'}


In [13]:
ray.shutdown()